In [1]:
#import modules
import json
import edge_validation as ev

In [2]:
# Add markdown comment
with open('configuration/belfast.json') as f:
    config = json.load(f)

In [3]:
# Add markdown comment
(graphml, osm_buffer, gdf_official) = ev.load_data('belfast.json', 'Belfast')

In [4]:
# Add markdown comment
(gdf_osm) = ev.refine_data(graphml, osm_buffer, gdf_official)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf3 in position 31: invalid continuation byte

In [ ]:
# Add markdown comment
(osm_data_clipped, official_clipped) = ev.clip_data

In [ ]:
# Add markdown comment
ev.plot_map(gdf_study_area)

In [ ]:
# Add markdown comment
ev.plot_map(gdf_osm)

In [ ]:
# Add markdown comment
ev.plot_map(gdf_official)

In [ ]:
# Add markdown comment
ev.plot_map(osm_data_clipped)

In [ ]:
# Add markdown comment
ev.plot_map(official_clipped)

In [ ]:
# Add markdown comment
ev.total_length(gdf_osm)

In [ ]:
# Add markdown comment
ev.total_length(gdf_official)

In [ ]:
# Add markdown comment
ev.buffer_intersected(5, official_buffer)

In [ ]:
# Add markdown comment
ev.buffer_intersected(5, osm_buffer)

In [ ]:
# Add markdown comment
ev.shared_area(5)